In [1]:
from scipy.ndimage.morphology import binary_fill_holes, binary_dilation, binary_erosion, generate_binary_structure
from skimage.morphology import label
import nrrd
import numpy as np

In [2]:
kidney6, _ = nrrd.read('results/2d_fill_or_kidney6.nrrd')
kidney9, _ = nrrd.read('results/2d_fill_or_kidney9.nrrd')

## Seperate vein and kidney

In [5]:
kidney6_dilated = binary_dilation(kidney6)
vein1 = kidney9-kidney6_dilated
vein1[np.where(vein1<0)] = 0
nrrd.write('results/vein1.nrrd', vein1.astype(np.float))

In [6]:
for i in range(5):
    kidney6_dilated = binary_dilation(kidney6_dilated)
vein2 = kidney9-kidney6_dilated
vein2[np.where(vein2<0)] = 0
nrrd.write('results/vein2.nrrd', vein2.astype(np.float))

In [12]:
new_mask, num = label(vein2, neighbors=4, background=None, return_num=True, connectivity=None)
order_ind = list()
for i in range(num):
    inds = np.where(new_mask==i)
    mask_size = inds[0].shape[0]
    order_ind.append((i, mask_size))

In [15]:
all_mask = np.zeros(new_mask.shape)
if len(order_ind)>1:
    sorted_ind = sorted(order_ind, key=lambda x: x[1], reverse=True)
    selected_inds = [sorted_ind[4]]
    for j in selected_inds:
        inds = np.where(new_mask==j[0])
        print(inds[0].shape)
        all_mask[inds] = 1

(14331,)


In [16]:
nrrd.write('results/vein2_cca.nrrd', all_mask.astype(np.float))

In [20]:
vein1_cca,_ = nrrd.read('results/vein1_cca.nrrd')
vein2_cca,_ = nrrd.read('results/vein2_cca.nrrd')

In [24]:
nrrd.write('results/all_vein.nrrd', (vein1_cca+vein2_cca).astype(np.float))

In [21]:
vein1_cca = binary_dilation(vein1_cca)
vein2_cca = binary_dilation(vein2_cca)
only_kidney9 = (1-vein1_cca)*(1-vein2_cca)*kidney9
nrrd.write('results/all_kidney9.nrrd', only_kidney9.astype(np.float))

## Fusion for artery, tiny vessel and vein

In [ ]:
artery, _ = nrrd.read('results/artery9.nrrd')
tiny_vessel, _ = nrrd.read('results/tiny_vessel.nrrd')
vessel = np.logical_or(np.logical_or(artery, tiny_vessel), vein1_cca+vein2_cca)
nrrd.write('results/all_vessel.nrrd', vessel.astype(np.float))

## Fusion for vessel and kidney

In [35]:
all_kidney_vessel = np.zeros(vessel.shape)
all_kidney_vessel[np.where(only_kidney9>0)] = 1
all_kidney_vessel[np.where(vessel>0)] = 1
nrrd.write('results/all_kidney_vessel.nrrd', all_kidney_vessel.astype(np.float))

In [43]:
index = np.where(vessel>0)
index = np.array(index).T
index.shape

(167250, 3)

In [44]:
with open('vessel_points.xyz', 'w') as wf:
    for ind in range(index.shape[0]):
        line = str(index[ind][0]) + ' ' + str(index[ind][1]) + ' ' + str(index[ind][2]) +'\n'
        wf.write(line)

## Fusion for vessel and kidney and tumor

In [ ]:
tumor9, _ = nrrd.read('results/tumor9.nrrd')
all_kidney_vessel, _ = nrrd.read('results/all_kidney_vessel.nrrd')
all_model = np.logical_or(all_kidney_vessel, tumor9)
nrrd.write('results/all_kidney_vessel_tumor.nrrd', all_model.astype(np.float))

In [ ]:
index = np.where(vessel>0)
index = np.array(index).T
index.shape

In [ ]:
with open('results/xyz/tumor_points.xyz', 'w') as wf:
    for ind in range(index.shape[0]):
        line = str(index[ind][0]) + ' ' + str(index[ind][1]) + ' ' + str(index[ind][2]) +'\n'
        wf.write(line)